In [4]:
from sentence_transformers import SentenceTransformer, util
import os
import json
import gzip

In [5]:
util.http_get('https://public.ukp.informatik.tu-darmstadt.de/reimers/sentence-transformers/datasets/simplewiki-2020-11-01.jsonl.gz', 'simplewiki-2020-11-01.jsonl.gz')

  0%|          | 0.00/50.2M [00:00<?, ?B/s]

In [11]:
model = SentenceTransformer('nq-distilbert-base-v1')

In [6]:
passages = []
with gzip.open('simplewiki-2020-11-01.jsonl.gz', 'rt', encoding='utf8') as file:
    for line in file:
        data = json.loads(line.strip())
        # print(data)
        for paragraph in data['paragraphs']:
            passages.append([data['title'], paragraph])
        # break

In [7]:
len(passages)

509663

In [8]:
from random import shuffle
shuffle(passages)

In [9]:
passages = passages[0:1_000]

In [12]:
corpus_embeddings = model.encode(passages, convert_to_tensor=True, show_progress_bar=True)

Batches:   0%|          | 0/32 [00:00<?, ?it/s]

In [13]:
def get_answer(query):

    question_embedding = model.encode(query, convert_to_tensor=True)
    hits = util.semantic_search(question_embedding, corpus_embeddings, top_k=3)[0]

    print("Results:")
    for hit in hits:
        print(passages[hit['corpus_id']])

In [18]:
get_answer("tell me about Boston city")

Results:
['Boston', 'Boston is the largest city of Massachusetts in the United States. It was founded in 1630. Boston is one of the oldest, richest and most culturally important cities in the United States.']
['City', 'In modern times many cities have grown bigger and bigger. The whole area is often called a ""metropolis"" and can sometimes includes several small ancient towns and villages. The "metropolis" of London includes the City of London, the City of Westminster and many old villages such as Notting Hill, Southwark, Richmond, Greenwich and etc. The part that is officially known as the "City of London" only takes up one square mile. The rest is known as "Greater London". Many other cities have grown in the same way.']
['Northern California', 'Northern California is a region of California. The definitions of it vary, but it usually contains the area around San Francisco and Sacramento. 10,000,000 people live in Northern California.']


In [19]:
get_answer("when did the cold war end?")

Results:
['Brigid Berlin', 'Berlin died of cardiac arrest from a pulmonary embolism in New York City on July 17, 2020 at the age of 80.']
['Ely Cathedral', "In 1986 it was discovered that the roofs, timber and stonework of much of the Cathedral were becoming unsafe. A public appeal was launched. £4,000,000 was needed to secure the Cathedral's future and to prevent its doors being closed for good. In just over a year, the money was given or promised. The Great Restoration was completed in the year 2000."]
['Susan Rice', 'Susan Elizabeth Rice (born November 17, 1964) is an American diplomat. She was the National Security Advisor from 2013 through 2017 during the Barack Obama presidency. Before, she was the United States Ambassador to the United Nations between 2009 to 2013.']
